In [1]:
import copy
from typing import Dict, Any
from pathlib import Path
import tempfile

from allennlp.common import Params
import target_extraction
from target_extraction.allen import AllenNLPModel
import lstm

import config
from target_extraction.analysis.sentiment_metrics import get_labels
from target_extraction.data_types import TargetTextCollection
import json

def predictions(model_name, split_name, dataset_name, preidction_collection, save_dir):
    result_fp = save_dir / f'{dataset_name} {split_name}.json'
    labels = get_labels(preidction_collection, 'target_sentiments', 'predicted_sentiment')
    with result_fp.open('w+') as result_file:
        json.dump(labels, result_file)

def add_word_vector(word_vector_name: str, params_dict: Dict[str, any], 
                    model_name: str, file_path: Path) -> Path:
    temp_params_dict = copy.deepcopy(params_dict)
    word_embedding_fp = None
    dimension = 0
    if word_vector_name == 'GloVe 300':
        word_embedding_fp = config.WORD_EMBEDDING_DIR / 'glove.840B.300d.txt'
        dimension = 300
    else:
        raise ValueError(f'word_vector_name {word_vector_name} is not one of accepted names.')
    assert word_embedding_fp is not None
    word_embedding_fp = str(word_embedding_fp)
    if model_name in ['TDLSTM', 'TCLSTM']:
        temp_params_dict['model']['context_field_embedder']['token_embedders']['tokens']['embedding_dim'] = dimension
        temp_params_dict['model']['context_field_embedder']['token_embedders']['tokens']['pretrained_file'] = word_embedding_fp
    elif model_name == 'LSTM':
        temp_params_dict['model']['embedder']['token_embedders']['tokens']['embedding_dim'] = dimension
        temp_params_dict['model']['embedder']['token_embedders']['tokens']['pretrained_file'] = word_embedding_fp
    else:
        raise ValueError(f'The model name {model_name} is not a recognised model name')
    if model_name == 'TDLSTM':
        temp_params_dict['model']['left_text_encoder']['hidden_size'] = dimension
        temp_params_dict['model']['left_text_encoder']['input_size'] = dimension
        temp_params_dict['model']['right_text_encoder']['hidden_size'] = dimension
        temp_params_dict['model']['right_text_encoder']['input_size'] = dimension
    elif model_name == 'TCLSTM':
        temp_params_dict['model']['left_text_encoder']['hidden_size'] = dimension
        temp_params_dict['model']['left_text_encoder']['input_size'] = dimension * 2
        temp_params_dict['model']['right_text_encoder']['hidden_size'] = dimension
        temp_params_dict['model']['right_text_encoder']['input_size'] = dimension * 2
        temp_params_dict['model']['target_encoder']['embedding_dim'] = dimension
    elif model_name == 'LSTM':
        temp_params_dict['model']['encoder']['hidden_size'] = dimension
        temp_params_dict['model']['encoder']['input_size'] = dimension
    else:
        raise ValueError(f'The model name {model_name} is not a recognised model name')
    #print(temp_params_dict)
    return Params(temp_params_dict).to_file(file_path)


# Mass evaluation predictions for the LSTM based methods

This is the notebook that creates the mass evaluation predictions for the [Tang et al. 2016](https://www.aclweb.org/anthology/C16-1311.pdf) LSTM based methods. The cell below creates the predictions for all datasets in their normal size.

In [2]:
%%capture
# datasets
patience_values = ["5", "10"]
dataset_names = ['YouTuBean', 'Dong', 'Election', 'Laptop', 'Mitchell', 'Restaurant']
for patience_value in patience_values:
    for dataset_name in dataset_names:
        train_dataset = TargetTextCollection.load_json(config.neural_dataset_dir / f'{dataset_name} train.json')
        validation_dataset = TargetTextCollection.load_json(config.neural_dataset_dir / f'{dataset_name} validation.json')
        test_dataset = TargetTextCollection.load_json(config.neural_dataset_dir / f'{dataset_name} test.json')
        for model_name in ['LSTM', 'TDLSTM', 'TCLSTM']:
            result_dir = config.RESULTS_DIR / 'Mass Evaluation' / f"patience {patience_value}" / f"{model_name}"
            result_dir.mkdir(parents=True, exist_ok=True)
            validation_result_fp = result_dir / f'{dataset_name} validation.json'
            test_result_fp = result_dir / f'{dataset_name} test.json'
            if validation_result_fp.exists() and test_result_fp.exists():
                continue

            val_copy = TargetTextCollection(copy.deepcopy(list(validation_dataset.values())))
            test_copy = TargetTextCollection(copy.deepcopy(list(test_dataset.values())))
            base_config = config.MODEL_CONFIG_DIR / f"patience {patience_value}" / f"{model_name}.jsonnet"
            predictor = 'target-sentiment'
            for i in range(6):
                with tempfile.NamedTemporaryFile() as temp_file:
                    temp_file_path = Path(temp_file.name)
                    add_word_vector('GloVe 300', Params.from_file(base_config).as_dict(), 
                                    model_name, temp_file_path)
                    a_model = AllenNLPModel(model_name, temp_file_path, predictor, save_dir=None)
                    a_model.fit(train_data=train_dataset, val_data=validation_dataset, test_data=test_dataset)
                    a_model.predict_into_collection(val_copy, {'sentiments': 'predicted_sentiment'}, batch_size=32)
                    a_model.predict_into_collection(test_copy, {'sentiments': 'predicted_sentiment'}, batch_size=32)
            predictions(model_name, 'validation', dataset_name, val_copy, result_dir)
            predictions(model_name, 'test', dataset_name, test_copy, result_dir)

In the cell below creates the prediction for all datasets trained on the much smaller sized training dataset. This smaller sized training set is the same size as the YouTuBean training dataset size.

In [3]:
%%capture
# datasets
dataset_names = ['Dong', 'Election', 'Laptop', 'Mitchell', 'Restaurant']
patience_value = "10"
for dataset_name in dataset_names:
    train_dataset = TargetTextCollection.load_json(config.neural_small_dataset_dir / f'{dataset_name} train.json')
    validation_dataset = TargetTextCollection.load_json(config.neural_small_dataset_dir / f'{dataset_name} validation.json')
    test_dataset = TargetTextCollection.load_json(config.neural_dataset_dir / f'{dataset_name} test.json')
    for model_name in ['LSTM', 'TDLSTM', 'TCLSTM']:
        result_dir = config.RESULTS_DIR / 'Mass Evaluation Small Dataset' / f"patience {patience_value}" / f"{model_name}"
        result_dir.mkdir(parents=True, exist_ok=True)
        validation_result_fp = result_dir / f'{dataset_name} validation.json'
        test_result_fp = result_dir / f'{dataset_name} test.json'
        if validation_result_fp.exists() and test_result_fp.exists():
            continue

        val_copy = TargetTextCollection(copy.deepcopy(list(validation_dataset.values())))
        test_copy = TargetTextCollection(copy.deepcopy(list(test_dataset.values())))
        base_config = config.MODEL_CONFIG_DIR / f"patience {patience_value}" / f"{model_name}.jsonnet"
        predictor = 'target-sentiment'
        for i in range(6):
            with tempfile.NamedTemporaryFile() as temp_file:
                temp_file_path = Path(temp_file.name)
                add_word_vector('GloVe 300', Params.from_file(base_config).as_dict(), 
                                model_name, temp_file_path)
                a_model = AllenNLPModel(model_name, temp_file_path, predictor, save_dir=None)
                a_model.fit(train_data=train_dataset, val_data=validation_dataset, test_data=test_dataset)
                a_model.predict_into_collection(val_copy, {'sentiments': 'predicted_sentiment'}, batch_size=32)
                a_model.predict_into_collection(test_copy, {'sentiments': 'predicted_sentiment'}, batch_size=32)
        predictions(model_name, 'validation', dataset_name, val_copy, result_dir)
        predictions(model_name, 'test', dataset_name, test_copy, result_dir)